In [1]:
# --------- EDUCATIONAL NOTES ----------
# We keep imports minimal and stable to match your environment:
# pandas 1.2.4 / numpy 1.19.5 / scipy 1.7.0 / matplotlib 3.3.4
# --------------------------------------

from pathlib import Path
import json
import numpy as np
import pandas as pd

In [35]:
# ---- Paths ----
DATA_RAW = Path("./data_raw")   # folder with your 57 CSVs
DATA_INT = Path("./data_int")   # where we'll write standardized outputs
DATA_INT.mkdir(parents=True, exist_ok=True)

In [36]:
# ---- Trading-window approximations (for later notebooks too) ----
W_1, M_1, Q_1, Y_1 = 5, 21, 63, 252

In [37]:
# ---- Canonical schema mapping ----
# Based on your ACI.csv sample; tweak if any file deviates.
RENAME = {
    "DATE": "date",
    "OPEN": "open",
    "HIGH": "high",
    "LOW": "low",
    "CLOSE": "close",
    "VOLUME": "volume",
    "SYMBOL": "symbol",
    # Optional fields we may preserve as needed:
    "VWAP": "vwap",
    "VALUE": "value",
    "NO OF TRADES": "trades",
    "SERIES": "series",
    "PREV. CLOSE": "prev_close",
    "LTP": "ltp",
    "52W H": "hi_52w",
    "52W L": "lo_52w",
}

In [38]:

# ---- Columns we actually need for core analytics ----
# Keep OHLCV + SYMBOL + SERIES + DATE; others can be carried optionally.
USECOLS = [c for c in ["DATE","SYMBOL","SERIES","OPEN","HIGH","LOW","CLOSE","VOLUME","VWAP","VALUE","NO OF TRADES","PREV. CLOSE","LTP","52W H","52W L"] if c in RENAME]
DATE_COL = "DATE"  # original date column name (upper in your files)

# ---- Types (keep it simple and numerically stable) ----
# We'll let pandas infer floats; we explicitly parse dates and enforce category for symbol/series.
PARSE_DATES = [DATE_COL]
SERIES_FILTER = {"EQ"}  # keep only cash equities

In [39]:
def load_one_csv(path: Path) -> pd.DataFrame:
    """
    Read a single NSE equity EOD CSV (from jugaad-data) and standardize:
    - Parse DATE to datetime index
    - Rename columns to canonical lowercase names
    - Filter SERIES to 'EQ' (cash segment)
    - Sort by date, drop duplicates
    """
    df = pd.read_csv(
        path,
        usecols=USECOLS,
        parse_dates=PARSE_DATES,
        infer_datetime_format=True,  # speeds up parsing on older pandas
        dayfirst=False,              # NSE date is usually YYYY-MM-DD
        engine="c",
        low_memory=False,
    )

    # Rename to canonical
    df = df.rename(columns=RENAME)

    # Enforce expected minimal columns
    required = {"date","symbol","series","open","high","low","close","volume"}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"{path.name}: missing required columns: {missing}")

    # Filter to EQ if SERIES exists
    if "series" in df.columns and SERIES_FILTER:
        df = df[df["series"].isin(SERIES_FILTER)]

    # Basic cleaning
    df = df.dropna(subset=["date","symbol","close"])
    df = df.sort_values("date").drop_duplicates(subset=["date","symbol"])
    df["symbol"] = df["symbol"].astype("category")
    if "series" in df.columns:
        df["series"] = df["series"].astype("category")

    # Index by date for time-series ops
    df = df.set_index("date")

    return df


In [40]:
def ingest_folder(data_dir: Path) -> dict:
    """
    Load all CSVs in data_dir. Returns:
      {
        'per_symbol': {symbol: DataFrame(OHLCV...)},
        'panel_close': DataFrame(Date x Symbols of close),
        'quality': dict (counts, missingness, coverage)
      }
    """
    files = sorted(data_dir.glob("*.csv"))
    if not files:
        raise FileNotFoundError(f"No CSV files found in {data_dir.resolve()}")

    # 2A) Read & standardize each CSV into a per-symbol dataframe
    per_symbol = {}
    q = {"files": len(files), "rows_total": 0, "symbols": [], "date_spans": {}}

    for f in files:
        df = load_one_csv(f)

        # Expect exactly one symbol per file; if more, group by symbol.
        symbols = df["symbol"].unique().tolist()
        if len(symbols) != 1:
            # If a file somehow contains >1 symbol, split & store each
            for sym, dfx in df.groupby("symbol"):
                per_symbol[str(sym)] = dfx.copy()
        else:
            per_symbol[str(symbols[0])] = df

    # 2B) Report coverage
    for sym, dfx in per_symbol.items():
        q["symbols"].append(sym)
        q["rows_total"] += len(dfx)
        if not dfx.empty:
            q["date_spans"][sym] = {
                "start": str(dfx.index.min().date()),
                "end": str(dfx.index.max().date()),
                "rows": int(len(dfx)),
            }

    # 2C) Build a wide close-price panel, aligned on common dates (inner join)
    closes = []
    for sym, dfx in per_symbol.items():
        s = dfx["close"].rename(sym)
        closes.append(s)

    # Join on the intersection of all trading days to avoid NA cascades
    panel_close = pd.concat(closes, axis=1, join="inner").sort_index()

    # Optional: drop any dates with zeros/negatives if needed
    bad_mask = (panel_close <= 0).any(axis=1)
    if bad_mask.any():
        panel_close = panel_close[~bad_mask]

    return {"per_symbol": per_symbol, "panel_close": panel_close, "quality": q}


In [ ]:
Quant Library Roadmap
├─ 0. Ingestion & Preparation
│  ├─ Data Loading            – Read OHLCV data from CSVs or Jugaad API  [pandas]
│  ├─ Schema Normalization    – Standardize columns (date, open, high, low, close, volume) [pandas]
│  └─ Calendar Alignment      – Align symbols on common trading days (inner join) [pandas]
│
├─ 1. Returns Engine
│  ├─ Compute Returns         – Daily simple and log returns per asset [pandas, numpy]
│  └─ Rolling Windows         – Rolling returns for 1W, 1M, 1Q, 1Y (5/21/63/252 days) [pandas]
│
├─ 2. Statistical Measures
│  ├─ Rolling Stats           – Rolling mean, stdev, and z‑scores for each window [pandas]
│  └─ Portfolio Aggregation   – Compute weighted portfolio returns and volatility [pandas, numpy]
│
├─ 3. Correlation & Beta
│  ├─ Pairwise Correlation    – Full‑sample & rolling correlation matrices [pandas]
│  └─ Beta Calculation        – β(stock←index) and reverse β via cov/var [pandas, numpy]
│
├─ 4. PDFs & Distributions
│  ├─ Empirical PDFs          – Histograms & KDEs of realised returns [scipy.stats, matplotlib]
│  └─ Parametric Fits         – Fit Normal and Student‑t distributions [scipy.stats]
│
├─ 5. Realised Volatility & EWMA
│  └─ EWMA Calculation        – Exponentially weighted vol (λ=0.94/0.97) and annualisation [pandas]
│
├─ 6. Future Returns Distribution
│  └─ EWMA Variance Forecasts – Use final EWMA variance to simulate future returns (Normal or t) [numpy, scipy.stats]
│
└─ 7. Portfolio Risk Analysis
   ├─ Portfolio Statistics    – Compute μ, σ, and annualised vol for any weight vector [numpy, pandas]
   └─ VaR & ES                – Calculate historical VaR/ES and parametric VaR/ES [numpy]


In [46]:
from pathlib import Path
import pandas as pd

# canonical rename mapping (adjust if your files differ)
RENAME = {
    "DATE":"date","OPEN":"open","HIGH":"high","LOW":"low","CLOSE":"close",
    "VOLUME":"volume","SYMBOL":"symbol","SERIES":"series",
    "PREV. CLOSE":"prev_close","LTP":"ltp","VWAP":"vwap","VALUE":"value",
    "NO OF TRADES":"trades","52W H":"hi_52w","52W L":"lo_52w"
}
USECOLS = list(RENAME.keys())    # columns we care about
SERIES_FILTER = {"EQ"}           # keep only cash equities

def load_one_csv_manual(csv_path: str) -> pd.DataFrame:
    """Load one CSV and standardize it."""
    # ensure only expected columns are read
    available_cols = pd.read_csv(csv_path, nrows=0).columns
    usecols = [c for c in USECOLS if c in available_cols]

    df = pd.read_csv(
        csv_path,
        usecols=usecols,
        parse_dates=["DATE"],
        infer_datetime_format=True,
        engine="c",
        low_memory=False,
    ).rename(columns=RENAME)

    if "series" in df.columns:
        df = df[df["series"].isin(SERIES_FILTER)]
    df = df.dropna(subset=["date","symbol","close"])
    df = df.sort_values("date").drop_duplicates(subset=["date","symbol"])
    df["symbol"] = df["symbol"].astype("category")
    if "series" in df.columns:
        df["series"] = df["series"].astype("category")
    return df.set_index("date")

# Example: list the files you want to import
file_list = [
    r"C:\Users\quantbase\Desktop\quant\data_raw\ACI.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\TATAELXSI.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\TATAPOWER.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\UJJIVANSFB.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\VBL.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\VINCOFE.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\WAAREERTL.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\WHIRLPOOL.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\ZOMATO.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\ADANIENT.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\ADANIPORTS.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\AEROFLEX.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\AETHER.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\AVALON.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\AXISBANK.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\BAJFINANCE.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\BANDHANBNK.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\BHEL.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\CASTROLIND.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\CDSL.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\CLEAN.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\COALINDIA.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\DATAPATTNS.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\DCXINDIA.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\DMART.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\FEDERALBNK.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\HAL.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\HAPPSTMNDS.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\HDFCBANK.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\HFCL.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\IDFCFIRSTB.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\IEX.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\INDIGO.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\INOXINDIA.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\IRCTC.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\IREDA.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\IRFC.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\JIOFIN.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\KELLTONTEC.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\KOTAKBANK.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\LATENTVIEW.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\MAZDOCK.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\MCX.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\MTARTECH.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\NEWGEN.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\PPL.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\PREMIERENE.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\PRINCEPIPE.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\RTNPOWER.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\RVNL.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\SBICARD.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\SBIN.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\SONACOMS.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\SPANDANA.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\SWIGGY.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\SYNGENE.csv",
    r"C:\Users\quantbase\Desktop\quant\data_raw\TATACONSUM.csv"
]

per_symbol = {}
for fp in file_list:
    dfi = load_one_csv_manual(fp)
    for sym, g in dfi.groupby("symbol"):
        per_symbol[str(sym)] = g

# Build a wide close panel
closes = [df["close"].rename(sym) for sym, df in per_symbol.items()]
prices_close = pd.concat(closes, axis=1, join="inner").sort_index()
prices_close = prices_close[(prices_close > 0).all(axis=1)]

# Save results
DATA_INT = Path(r"C:\Users\quantbase\Desktop\quant\data_int")
DATA_INT.mkdir(parents=True, exist_ok=True)

prices_close.to_csv(DATA_INT / "prices_close.csv")
prices_close.to_pickle(DATA_INT / "prices_close.pkl")

# Long-form OHLCV
ohlcv_long = pd.concat(
    [df.assign(symbol=str(sym)) for sym, df in per_symbol.items()], axis=0
).sort_index()
ohlcv_long.to_pickle(DATA_INT / "ohlcv_long.pkl")

print("Manual ingestion complete. Symbols:", len(per_symbol), "Dates:", len(prices_close))


Manual ingestion complete. Symbols: 58 Dates: 0


In [74]:
ohlcv_long.head()

,series,open,high,low,prev_close,ltp,close,vwap,hi_52w,lo_52w,volume,value,trades,symbol
date,,,,,,,,,,,,,,
2022-03-30,EQ,154.20,155.25,151.55,151.55,152.45,152.45,153.59,194.70,115.65,34598,5.313847e+06,1260,PPL
2022-03-30,EQ,1078.80,1130.00,1060.20,1060.20,1067.00,1075.25,1094.41,1580.00,523.10,655141,7.169924e+08,39021,HAPPSTMNDS
2022-03-30,EQ,750.00,782.75,749.00,743.20,765.50,765.85,770.45,889.00,614.00,8328100,6.416402e+09,176864,TATACONSUM
2022-03-30,EQ,499.30,500.40,492.30,495.00,493.00,494.30,496.58,549.00,321.30,16461225,8.174331e+09,196570,SBIN
2022-03-30,EQ,240.95,244.90,237.75,239.10,241.10,241.70,241.20,267.85,89.90,22893839,5.521913e+09,134866,TATAPOWER


In [52]:
prices_close.head()

,ACI,TATAELXSI,TATAPOWER,UJJIVANSFB,VBL,VINCOFE,WAAREERTL,WHIRLPOOL,ETERNAL,ZOMATO,...,PRINCEPIPE,RTNPOWER,RVNL,SBICARD,SBIN,SONACOMS,SPANDANA,SWIGGY,SYNGENE,TATACONSUM
date,,,,,,,,,,,,,,,,,,,,,


In [53]:
# ---- EDUCATIONAL PURPOSES ----
# This summarizes how many rows (days) each symbol has,
# and their start/end dates. If the min/max periods don't overlap
# across many names, an inner join will be empty.
# ------------------------------

import pandas as pd

cov = []
for sym, dfx in per_symbol.items():
    if dfx.empty: 
        cov.append((sym, 0, None, None))
    else:
        cov.append((sym, len(dfx), dfx.index.min().date(), dfx.index.max().date()))

coverage = pd.DataFrame(cov, columns=["symbol","rows","start","end"]).sort_values("rows", ascending=False)
display(coverage.head(10))      # most history
display(coverage.tail(10))      # least history

print("Total symbols:", len(coverage))
print("Non-empty symbols:", (coverage["rows"]>0).sum())
print("Global start:", coverage["start"].min())
print("Global end:", coverage["end"].max())


,symbol,rows,start,end
29,HDFCBANK,856,2022-03-30,2025-09-09
40,KOTAKBANK,856,2022-03-30,2025-09-09
27,HAL,856,2022-03-30,2025-09-09
28,HAPPSTMNDS,856,2022-03-30,2025-09-09
1,TATAELXSI,856,2022-03-30,2025-09-09
30,HFCL,856,2022-03-30,2025-09-09
31,IDFCFIRSTB,856,2022-03-30,2025-09-09
32,IEX,856,2022-03-30,2025-09-09
33,INDIGO,856,2022-03-30,2025-09-09
35,IRCTC,856,2022-03-30,2025-09-09


,symbol,rows,start,end
14,AVALON,597,2023-04-18,2025-09-09
12,AEROFLEX,503,2023-08-31,2025-09-09
38,JIOFIN,501,2023-09-04,2025-09-09
36,IREDA,443,2023-11-29,2025-09-09
34,INOXINDIA,427,2023-12-21,2025-09-09
47,PREMIERENE,254,2024-09-03,2025-09-09
5,VINCOFE,222,2024-10-18,2025-09-09
55,SWIGGY,204,2024-11-13,2025-09-09
8,ETERNAL,104,2025-04-09,2025-09-09
6,WAAREERTL,104,2025-04-09,2025-09-09


Total symbols: 58
Non-empty symbols: 58
Global start: 2022-03-30
Global end: 2025-09-09


In [54]:
# Ensure all indices are pure dates (no time) and sorted
for sym, dfx in per_symbol.items():
    dfx.index = pd.to_datetime(dfx.index).normalize()
    per_symbol[sym] = dfx.sort_index()


In [55]:
bad = []
for sym, dfx in per_symbol.items():
    # force numeric, coerce errors to NaN
    dfx["close"] = pd.to_numeric(dfx["close"], errors="coerce")
    # flag symbols with too many missing/zero values
    if dfx["close"].le(0).sum() > 0 or dfx["close"].isna().sum() > 0:
        bad.append(sym)
print("Symbols with non-positive or missing closes:", bad[:10], "… total:", len(bad))



Symbols with non-positive or missing closes: [] … total: 0


In [56]:
# Intersect all date indexes progressively and show when it collapses
from functools import reduce

all_dates = [df.index.unique() for df in per_symbol.values() if not df.empty]
if not all_dates:
    print("All symbols empty—unexpected.")
else:
    inter = reduce(lambda a,b: a.intersection(b), all_dates)
    print("Size of full intersection:", len(inter))

    # Optional: find the smallest subset that still keeps the intersection non-empty
    # (sorted by history length)
    syms_sorted = coverage.sort_values("rows", ascending=False)["symbol"].tolist()
    inter2 = per_symbol[syms_sorted[0]].index.unique()
    cutoff_at = None
    for i, sym in enumerate(syms_sorted[1:], start=2):
        inter2 = inter2.intersection(per_symbol[sym].index.unique())
        if len(inter2) == 0:
            cutoff_at = i
            break
    print("Intersection becomes empty after combining ~", cutoff_at, "symbols (longest first).")


Size of full intersection: 0
Intersection becomes empty after combining ~ 57 symbols (longest first).


In [57]:
MIN_DAYS = 252  # adjust to your needs (1 year ~ 252 trading days)

core_syms = coverage.query("rows >= @MIN_DAYS")["symbol"].tolist()
print("Core symbols:", len(core_syms))

# Build the close panel on core universe; still inner-join, but now realistic
closes_core = [per_symbol[s]["close"].rename(s) for s in core_syms]
prices_close = pd.concat(closes_core, axis=1, join="inner").sort_index()

# Final clean (no non-positive)
prices_close = prices_close[(prices_close > 0).all(axis=1)]

print("Core panel shape:", prices_close.shape)  # (dates, symbols)


Core symbols: 54
Core panel shape: (126, 54)


In [58]:
MIN_DAYS = 252  # adjust to your needs (1 year ~ 252 trading days)

core_syms = coverage.query("rows >= @MIN_DAYS")["symbol"].tolist()
print("Core symbols:", len(core_syms))

# Build the close panel on core universe; still inner-join, but now realistic
closes_core = [per_symbol[s]["close"].rename(s) for s in core_syms]
prices_close = pd.concat(closes_core, axis=1, join="inner").sort_index()

# Final clean (no non-positive)
prices_close = prices_close[(prices_close > 0).all(axis=1)]

print("Core panel shape:", prices_close.shape)  # (dates, symbols)


Core symbols: 54
Core panel shape: (126, 54)


In [59]:
# Outer join across all symbols
closes_all = [df["close"].rename(sym) for sym, df in per_symbol.items()]
prices_close_outer = pd.concat(closes_all, axis=1, join="outer").sort_index()

# Only keep rows (dates) where we have at least K symbols present
K = 30  # pick a threshold that fits your analysis
enough_symbols = prices_close_outer.count(axis=1) >= K
prices_close = prices_close_outer.loc[enough_symbols]

# Keep only the columns (symbols) that are reasonably present on those dates
MINDATE_RATIO = 0.9  # symbol must be present on at least 90% of the kept dates
min_non_na = int(MINDATE_RATIO * len(prices_close))
cols_keep = prices_close.columns[prices_close.notna().sum(axis=0) >= min_non_na]
prices_close = prices_close[cols_keep]

# Drop any remaining NaNs (should be few after filtering)
prices_close = prices_close.dropna(how="any")
prices_close = prices_close[(prices_close > 0).all(axis=1)]

print("Thresholded panel shape:", prices_close.shape)


Thresholded panel shape: (573, 44)


In [60]:
from pathlib import Path
DATA_INT = Path(r"C:\Users\quantbase\Desktop\quant\data_int")
DATA_INT.mkdir(parents=True, exist_ok=True)

print("Panel dates:", prices_close.index.min(), "→", prices_close.index.max())
print("Panel shape (dates x symbols):", prices_close.shape)
print("Any NaNs?", prices_close.isna().any().any())

prices_close.to_csv(DATA_INT / "prices_close.csv")
prices_close.to_pickle(DATA_INT / "prices_close.pkl")


Panel dates: 2022-06-03 00:00:00 → 2025-09-09 00:00:00
Panel shape (dates x symbols): (573, 44)
Any NaNs? False


In [67]:
print (prices_close)

            TATAELXSI  TATAPOWER  UJJIVANSFB      VBL  WHIRLPOOL  ADANIENT  \
date                                                                         
2022-06-03    8433.30     231.05       16.30  1100.45    1635.15   2190.00   
2022-06-06    8696.25     232.25       16.20   733.05    1597.30   2225.05   
2022-06-07    8618.35     234.75       16.70   743.10    1585.75   2227.55   
2022-06-08    8608.95     233.00       16.35   744.75    1581.50   2177.20   
2022-06-09    8693.60     233.05       16.30   740.00    1578.15   2219.20   
...               ...        ...         ...      ...        ...       ...   
2025-09-03    5427.50     389.15       43.05   505.25    1344.90   2288.70   
2025-09-04    5428.00     382.60       43.11   489.50    1349.40   2278.80   
2025-09-05    5451.00     385.80       42.57   469.65    1357.00   2281.40   
2025-09-08    5469.50     383.25       44.17   474.05    1363.00   2310.90   
2025-09-09    5553.50     384.65       47.47   474.50    1371.20